Aim: predict age and gender from RFIW2021 data -- train and val sets.

1. Inspect MiVOLO to understand its setup
2. Copy the necessary code
3. Adapt it to save each prediction to a dataframe (maybe extending val.txt to have age and gender)
4. Analyse the generated dataframe

## Import libraries

In [5]:
from dataclasses import dataclass
from pathlib import Path

import cv2
import pandas as pd
from mivolo.data.data_reader import get_all_files
from mivolo.predictor import Predictor
from tqdm import tqdm

# Set up paths

In [7]:
try:
    IS_NOTEBOOK = True
    HERE = Path(__file__).resolve().parent
except NameError:
    IS_NOTEBOOK = False
    HERE = Path().resolve()

DATA_DIR = Path(HERE, "../rfiw2021/Track1/Validation/val-faces")
MODELS_DIR = Path(HERE, "../MiVOLO/models")

In [10]:
@dataclass
class Config:
    device: str = "cuda"
    detector_weights: str = MODELS_DIR / "yolov8x_person_face.pt"
    checkpoint: str = MODELS_DIR / "mivolo_imbd.pth.tar"
    with_persons: bool = False
    disable_faces: bool = False
    draw = False

In [41]:
args = Config()
predictor = Predictor(args)

Model summary (fused): 268 layers, 68125494 parameters, 0 gradients


In [43]:
image_files = get_all_files(DATA_DIR)

preds = []

for img_p in tqdm(image_files[:10]):
    img = cv2.imread(img_p)
    pred, out_im = predictor.recognize(img)
    img_info = {k: v for k, v in zip(["fid", "mid", "pid_filename"], Path(img_p).parts[-3:])}
    pred_info = {"age": pred.ages[1], "gender": pred.genders[1], "gender_score": pred.gender_scores[1]}
    preds.append({**img_info, **pred_info})

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 25.95it/s]


In [44]:
preds

[{'fid': 'F0119',
  'mid': 'MID2',
  'pid_filename': 'P01237_face3.jpg',
  'age': 19.8,
  'gender': 'female',
  'gender_score': 0.99755859375},
 {'fid': 'F0119',
  'mid': 'MID2',
  'pid_filename': 'P01243_face1.jpg',
  'age': 18.37,
  'gender': 'female',
  'gender_score': 0.9970703125},
 {'fid': 'F0119',
  'mid': 'MID2',
  'pid_filename': 'P01236_face0.jpg',
  'age': 20.42,
  'gender': 'female',
  'gender_score': 0.99755859375},
 {'fid': 'F0119',
  'mid': 'MID2',
  'pid_filename': 'P01240_face2.jpg',
  'age': 26.59,
  'gender': 'female',
  'gender_score': 0.9970703125},
 {'fid': 'F0119',
  'mid': 'MID2',
  'pid_filename': 'P01239_face3.jpg',
  'age': 23.44,
  'gender': 'female',
  'gender_score': 0.9970703125},
 {'fid': 'F0119',
  'mid': 'MID2',
  'pid_filename': 'P01241_face1.jpg',
  'age': 19.31,
  'gender': 'female',
  'gender_score': 0.99755859375},
 {'fid': 'F0119',
  'mid': 'MID2',
  'pid_filename': 'P01234_face0.jpg',
  'age': 20.35,
  'gender': 'female',
  'gender_score': 0.997

In [45]:
df = pd.DataFrame(preds)

In [46]:
df

,fid,mid,pid_filename,age,gender,gender_score
0,F0119,MID2,P01237_face3.jpg,19.80,female,0.997559
1,F0119,MID2,P01243_face1.jpg,18.37,female,0.997070
2,F0119,MID2,P01236_face0.jpg,20.42,female,0.997559
3,F0119,MID2,P01240_face2.jpg,26.59,female,0.997070
4,F0119,MID2,P01239_face3.jpg,23.44,female,0.997070
5,F0119,MID2,P01241_face1.jpg,19.31,female,0.997559
6,F0119,MID2,P01234_face0.jpg,20.35,female,0.997559
7,F0119,MID2,P01242_face1.jpg,19.04,female,0.997559
8,F0119,MID2,P01235_face0.jpg,22.73,female,0.997070
9,F0119,MID2,P01238_face0.jpg,30.78,female,0.997070
